In [ ]:
import pandas as pd

# Load att df

In [ ]:
playa_att_csv_path = '../data/playa_nogeometry_whucs.csv'
playa_att_df = pd.read_csv(playa_att_csv_path).set_index('id')

In [ ]:
# Convert HUC and author to int codes: (using huc8 instead of huc12)
playa_att_df['huc8'] = playa_att_df['huc12'].astype(str).str[0:8]
playa_att_df['huc8'] = playa_att_df['huc8'].astype('category').cat.codes

playa_att_df['author'] = playa_att_df['author'].astype('category').cat.codes

In [ ]:
# # Calculate lookup tables for HUC code (using HUC8)
# playa_att_df['huc8'] = playa_att_df['huc12'].astype(str).str[0:8]
# huc8_ascat = playa_att_df['huc8'].astype(int).astype('category')
# cats = huc8_ascat.cat.categories
# codes = huc8_ascat.cat.codes.unique().tolist()
# huc_dict = dict(zip(cats,codes))
# with open('huc_lut.json', 'w') as fp:
#     json.dump(huc_dict, fp)
    
# # Calculate lookup tables for author string
# author_ascat = playa_att_df['author'].astype('category')
# cats = author_ascat.cat.categories
# codes = author_ascat.cat.codes.unique().tolist()
# author_dict = dict(zip(cats,codes))
# with open('author_lut.json', 'w') as fp:
#     json.dump(author_dict, fp)

# Preprocess data to split up by state and county

In [ ]:
inun_csv_path = '../data/jrc-water_1984-2019.csv'

In [ ]:
inun_df = pd.read_csv(inun_csv_path).set_index('id').drop('.geo', axis=1).rename(columns={'mean':'inundation'})

In [ ]:
def save_state_county_csvs_inun(state, inun_df, playa_att_df):
    # Join state and county
    state_df = inun_df.join(playa_att_df.loc[playa_att_df['state']==state,
                                             ['state', 'countyfips', 'acres','huc8', 'winddevyr',
                                              'author','meanwetfrq','cluster','healthy','farmed','hydromod',
                                              'fttoroad', 'sthick2013']
                                            ]
                            ,how='inner')
    
    # Split up weird index into year + months
    split_index = state_df['system:index'].str.split('_', expand=True)
    state_df = state_df.assign(year=split_index[0].astype('int16'),
                               month=split_index[1].astype('int16'),
                               area=state_df['acres']*state_df['inundation'])
    state_df.drop(['system:index', 'state'], axis=1, inplace=True)
    
    # Save state csv
    state_out_path = '../data/state_county_csvs/inun_frac_{}.csv'.format(state)
    state_df.drop('countyfips', axis=1).to_csv(state_out_path)
    
    # Save county csvs
    for county in state_df['countyfips'].unique():
        county_df = state_df.loc[state_df['countyfips']==county]
        county_out_path = '../data/state_county_csvs/counties/inun_frac_{}_{}.csv'.format(state, county)
        county_df.drop('countyfips', axis=1).to_csv(county_out_path)

In [ ]:
# Save state and county csvs
for state in playa_att_df['state'].unique():
    save_state_county_csvs_inun(state, inun_df, playa_att_df)


# Preprocess and split weather data

## Prism data (current version)

In [ ]:
weather_csv_path = '../data/prism.csv'

In [ ]:
weather_df = pd.read_csv(weather_csv_path).set_index('id').drop('.geo', axis=1)

In [ ]:
def save_state_county_csvs_weather(state, weather_df, playa_att_df):
    # Join state and county
    state_df = weather_df.join(playa_att_df.loc[playa_att_df['state']==state,
                                             ['state', 'countyfips']],how='inner')
    
    # Split up weird index into year + months, little different than inundation
    state_df = state_df.assign(year=state_df['system:index'].str.slice(0,4).astype('int16'),
                               month=state_df['system:index'].str.slice(4,6).astype('int16'))
    state_df.drop(['system:index', 'state'], axis=1, inplace=True)
    
    # Rename to simpler column names
    state_df = state_df.rename(columns={'tmean':'temp', 'ppt':'precip', 'vpdmin':'vpd'})
    
    # Save state csv
    state_out_path = '../data/state_county_csvs/weather_{}.csv'.format(state)
    state_df.drop('countyfips', axis=1).to_csv(state_out_path)
    
    # Save county csvs
    for county in state_df['countyfips'].unique():
        county_df = state_df.loc[state_df['countyfips']==county]
        county_out_path = '../data/state_county_csvs/counties/weather_{}_{}.csv'.format(state, county)
        county_df.drop('countyfips', axis=1).to_csv(county_out_path)

In [ ]:
# Save state and county csvs
for state in playa_att_df['state'].unique():
    save_state_county_csvs_weather(state, weather_df, playa_att_df)


## Era Data (outdated, using prism now)

In [ ]:
# weather_csv_path = '../data/era5_1984-2019.csv'

In [ ]:
# weather_df = pd.read_csv(weather_csv_path).set_index('id').drop('.geo', axis=1)

In [ ]:
# # Save state and county csvs
# for state in playa_att_df['state'].unique():
    
#     # Join state and county
#     state_df = weather_df.join(playa_att_df.loc[playa_att_df['state']==state,
#                                              ['state', 'countyfips']],how='inner')
    
#     # Split up weird index into year + months, little different than inundation
#     state_df = state_df.assign(year=state_df['system:index'].str.slice(0,4).astype('int16'),
#                                month=state_df['system:index'].str.slice(4,6).astype('int16'))
#     state_df.drop(['system:index', 'state'], axis=1, inplace=True)
    
#     # Rename to simpler column names
#     state_df = state_df.rename(columns={'mean_2m_air_temperature':'temp', 'total_precipitation':'precip'})
    
#     # Save state csv
#     state_out_path = '../data/state_county_csvs/weather_{}.csv'.format(state)
#     state_df.drop('countyfips', axis=1).to_csv(state_out_path)
    
#     # Save county csvs
#     for county in state_df['countyfips'].unique():
#         county_df = state_df.loc[state_df['countyfips']==county]
#         county_out_path = '../data/state_county_csvs/counties/weather_{}_{}.csv'.format(state, county)
#         county_df.drop('countyfips', axis=1).to_csv(county_out_path)